In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [ ]:
# -----------------------------------------------------------------------------
# SECTION 1: SETUP AND DATA LOADING
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Deep Learning Imports
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Evaluation Imports (using scikit-learn for metrics)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize NLTK components
lemmatizer =
STOPWORDS =
PUNCTUATION =

# --- File Paths ---
TRAIN_PATH =
TEST_PATH =
SUB_PATH =

print("1. Loading datasets...")
try:
    df_train = pd.read_csv(TRAIN_PATH)
    df_test = pd.read_csv(TEST_PATH)
    df_submission = pd.read_csv(SUB_PATH)
    print("   Data loaded successfully.")
except FileNotFoundError as e:
    print(f"   Error loading data: {e}. Please ensure files are in the correct path.")
    exit()

2025-10-16 13:41:46.772065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760622107.072744      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760622107.156546      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


1. Loading datasets...
   Data loaded successfully.


In [ ]:
# -----------------------------------------------------------------------------
# SECTION 2: EXPLORATORY DATA ANALYSIS
# -----------------------------------------------------------------------------
print("\n2. Initial Data Exploration:")
print(f"   Training Data Shape: {}")
print(f"   Test Data Shape: {}")
print("\n--- Training Data Head ---")
print()

target_counts =
print("\n--- Target Distribution (0=Not Disaster, 1=Disaster) ---")
print(target_counts)


2. Initial Data Exploration:
   Training Data Shape: (7613, 5)
   Test Data Shape: (3263, 4)

--- Training Data Head ---
   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  

--- Target Distribution (0=Not Disaster, 1=Disaster) ---
target
0    4342
1    3271
Name: count, dtype: int64


In [ ]:
# -----------------------------------------------------------------------------
# SECTION 3: TEXT PREPROCESSING & CLEANING (Slightly modified comments)
# -----------------------------------------------------------------------------
print("\n3. Cleaning and Preprocessing Text Data...")

def clean_text(text):
    """Performs comprehensive text cleaning for tweets."""
    # Normalize
    text =
    # Remove URLs ( Hint, use : 'https?://\S+|www\.\S+')
    text =
    # Remove HTML special characters ( Hint, use : '&[a-z]+;')
    text =
    # Remove user mentions ( Hint, use : '@\w+')
    text =
    # Remove punctuation
    text =

    # Tokenization, Stopword Removal, and Lemmatization
    tokens =

    return

# Apply cleaning
df_train['clean_text'] =
df_test['clean_text'] =

print("   Sample of cleaned text:")
print(df_train['clean_text'].head().to_string())

In [ ]:
# -----------------------------------------------------------------------------
# SECTION 4: DEEP LEARNING DATA PREPARATION (Tokenization & Padding)
# -----------------------------------------------------------------------------
print("\n4. Deep Learning Data Preparation (Tokenization & Padding)...")

# Define our features (X) and target (y)
X = df_train['clean_text']
y = df_train['target'].values # Convert to numpy array for Keras

# --- Hyperparameters for Tokenization ---
VOCAB_SIZE =                    # Max number of unique words to keep. Try 10000, 20000, 30000
MAX_LENGTH =                    # Max sequence length (words per tweet). Try 20, 30, 40
OOV_TOKEN = "<oov>"             # Token for Out-Of-Vocabulary words

# 1. Initialize and Fit Tokenizer
# The tokenizer learns the vocabulary from the training text
tokenizer =
tokenizer.fit_on_texts(X)

# 2. Convert text to sequences of integers
train_sequences =
test_sequences =

# 3. Padding sequences
# Pad all sequences to MAX_LENGTH, padding = 'post', truncating = 'post'
X_padded =
X_test_padded =

print(f"   Padded Sequence Shape (Train): {}")
print(f"   Padded Sequence Shape (Test): {}")
print(f"   Vocabulary Size used: {}")

In [ ]:
# -----------------------------------------------------------------------------
# SECTION 5: DEEP LEARNING MODEL DEFINITION, TRAINING, AND EVALUATION
# -----------------------------------------------------------------------------
print("\n5. Deep Learning Model Training and Evaluation...")

# --- Splitting Data for Validation ---
X_train, X_val, y_train, y_val = train_test_split(
    X_padded, y,
    test_size= , #  Try the split sizes 0.1, 0.2, 0.3
    random_state=42
)

print(f"   Training on {X_train.shape[0]} samples, Validating on {X_val.shape[0]} samples.")

# --- Hyperparameters for DL Model ---
EMBEDDING_DIM =     # Size of the word vector representation. Try 50, 100, 200
LSTM_UNITS =        # Number of hidden units in the LSTM layer. Try 32, 64, 128
EPOCHS =            # Number of full passes over the training data. Start low (5), try higher (20)
BATCH_SIZE =        # Number of samples per gradient update. Try 16, 32, 64

# --- Define the Sequential Model ---
def create_lstm_model(vocab_size, embedding_dim, max_length, lstm_units):
    model = Sequential([
        # Layer 1: Embedding Layer
        # Converts integer sequences to dense vectors (word embeddings)


        # Layer 2: LSTM Layer
        # Processes sequences, capturing long-term dependencies in the text


        # Layer 3: Output Layer
        # Single neuron with sigmoid activation for binary classification (0 or 1)

    ])

    # Compile the model
    model.compile(
        loss= , # Appropriate loss for binary classification
        optimizer= ,           # Use a default optimizer
        metrics=['accuracy']
    )
    return model

# Initialize and summarize the model
dl_model = create_lstm_model(VOCAB_SIZE, EMBEDDING_DIM, MAX_LENGTH, LSTM_UNITS)
print("\n--- Deep Learning Model Summary ---")
dl_model.summary()
#

# --- Train the Model ---
print("\n   Starting model training...")
history = dl_model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    verbose=1 # Display training progress
)
print("   Model training complete.")

# --- Evaluate on Validation Set ---
# Keras predictions output probabilities (0.0 to 1.0)
y_val_pred_proba =
# Convert probabilities to binary classes (0 or 1) using a threshold of 0.5
y_val_pred =

# Calculate key metrics
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

print(f"\n--- Validation Results ---")
print(f"   Accuracy: {val_accuracy:.4f}")
print(f"   F1 Score (Target Metric): {val_f1:.4f}")
print("\n--- Detailed Classification Report ---")
# Flatten y_val_pred for compatibility with classification_report
print(classification_report(y_val, y_val_pred.flatten(), target_names=['Not Disaster (0)', 'Disaster (1)']))


In [ ]:
# -----------------------------------------------------------------------------
# SECTION 6: GENERATE FINAL PREDICTIONS FOR SUBMISSION
# -----------------------------------------------------------------------------
print("\n6. Generating Final Predictions...")

# Predict probabilities on the test set
final_predictions_proba =
# Convert probabilities to binary (0 or 1)
final_predictions =
# Flatten the 2D array to a 1D series
final_predictions = final_predictions.flatten()

# Create the submission dataframe
df_submission['target'] = final_predictions

# Ensure the submission file only has the required columns: id and target
submission_output = df_submission[['id', 'target']]

# Save the submission file
submission_output.to_csv('submission_lstm.csv', index=False)
print("   Submission file 'submission_lstm.csv' created successfully.")
print("\n--- Sample Submission Head ---")
print(submission_output.head())

In [ ]:
# =============================================================================
# CLASS WRAP-UP: EXPERIMENTS TO TRY
# =============================================================================
# 1. MODEL ARCHITECTURE (Section 5):
#    - Change the recurrent layer: Replace LSTM(64) with GRU(64) (GRU is generally faster).
#    - Add Dropout: Insert a Dropout layer (e.g., Dropout(0.5)) after the Embedding layer to prevent overfitting.
#    - Try Bidirectional: Wrap the LSTM layer in a Bidirectional layer (tf.keras.layers.Bidirectional(LSTM(64))).
#
# 2. HYPERPARAMETER TUNING (Section 4 & 5):
#    - Adjust embedding_dim: Try EMBEDDING_DIM = 50 or 200.
#    - Adjust LSTM_UNITS: Try LSTM_UNITS = 32 or 128.
#    - Adjust EPOCHS: See how the training and validation loss/accuracy change after 5, 10, or 20 epochs.
#
# 3. DATA PREPARATION (Section 4):
#    - Adjust sequence length: Try MAX_LENGTH = 20 or 40.
#
# This Deep Learning pipeline should yield a competitive score!
# Happy coding!